In [7]:
import json
import pandas as pd

In [ ]:
def returnValueFromData(data, key):
    temp = data.copy()
    for i in key.split('.'): #[legacy, created_at]
        if i == "urls" or i == "pinned_tweet_ids_str" or i == "media":
            if len(temp[i])>0:
                temp = temp[i][0]
            else:
                return None
        elif i in temp.keys():
            temp = temp[i]
        else:
            return None
    return temp

with open("MrBeast_user_profile_info.json", "r") as json_file:
        data = json.load(json_file)
        userprofile_key_to_key_mapping = {
            "user_or_tweet": "__typename",
            "is_blue_verified": "core.user_results.result.is_blue_verified",
            "created_at": "core.user_results.result.legacy.created_at",
            "description": "core.user_results.result.legacy.description",
            "user_web_url": "core.user_results.result.legacy.entities.url.urls.display_url",
            "expanded_user_web_url": "core.user_results.result.legacy.entities.url.urls.expanded_url",
            "favourites_count": "core.user_results.result.legacy.favourites_count",
            "followers_count": "core.user_results.result.legacy.followers_count",
            "following": "core.user_results.result.legacy.friends_count",
            "has_custom_timelines": "core.user_results.result.legacy.has_custom_timelines",
            "location": "core.user_results.result.legacy.location",
            "photos_and_videos_count": "core.user_results.result.legacy.media_count",
            "name": "core.user_results.result.legacy.name",
            "pinned_tweet_ids_str": "core.user_results.result.legacy.pinned_tweet_ids_str",
            "profile_banner_url": "core.user_results.result.legacy.profile_banner_url",
            "profile_image_url": "core.user_results.result.legacy.profile_image_url_https",
            "screen_name": "core.user_results.result.legacy.screen_name",
            "posts_count": "core.user_results.result.legacy.statuses_count",
            "professional_type": "core.user_results.result.professional.professional_type",
            "verified_phone_status": "core.user_results.result.verified_phone_status",
            "is_edit_eligible": "edit_control.is_edit_eligible",
            "edits_remaining": "edit_control.edits_remaining",
            "views_count": "views.count",
            "source": "source",
            "post_created_at": "legacy.created_at",
            "post_conversation_id_str": "legacy.conversation_id_str",
            "hashtags": "legacy.entities.hashtags",
            "media_url_https": "legacy.entities.media.media_url_https",
            "media_type": "legacy.entities.media.media_url_https",
            "user_mentions_in_media": "legacy.entities.media.user_mentions",
            "favorite_count": "legacy.favorite_count",
            "full_text_in_post": "legacy.full_text",
            "language_of_the_post": "legacy.lang",
            "quote_count": "legacy.quote_count",
            "reply_count": "legacy.reply_count",
            "retweet_count": "legacy.retweet_count",
            "is_retweeted": "legacy.retweeted",
        }

userprofile_json = {}
        
for i in userprofile_key_to_key_mapping.items():
    userprofile_json[i[0]] = [returnValueFromData(data, i[1])]
    
user_df = pd.DataFrame(userprofile_json)
user_df.head()